# Análise de Dados e Inteligência Artificial - NHL

<p align="center">
    <img width=650 src="https://github.com/DanOliveira-DS/NHL/blob/master/images/web_hockey_basen_art.jpg?raw=true">
</p>

Muito inspirado por Michael Lewis e seu livro "*Moneyball: The Art of Winning an Unfair Game*" (leitura obrigatória para amantes da area que foi adaptado para o cinema com o título \"O Homem que Mudou o Jogo\", do diretor Bennett Miller) mostrou como a análise de dados aliado a um plano de jogo podem ser uma combinação fantástica. Nos últimos anos os avanços na tecnologia (dispositivos e sensores wearable como pulseiras eletrônicas, coletes, dispositivos GPS e outras tecnologias inteligentes) nos permitiram coletar cada vez mais dados. Entretanto, um monte de dados acumulados não são úteis se você não consegue interpretá-los. Portanto, tão importante quanto coletar dados estatísticos é saber decifrá-los. Com isso, a ciência de dados aplicada aos esportes já é uma realidade porque agora somos capazes de gerar valor através desses dados ajudando assim nas tomadas de decisão.

A análise desses novos dados possibilitou não apenas verificar estatísticas clássicas (vitorias, derrotas, gols feitos, gols sofridos..), mas tambem as estatísticas avançadas que procuram entender as diferentes maneiras com que uma jogada ou jogador pode influenciar uma partida. Com isso os times passaram a ser mais competitivos, sendo capazes de se prepararem  para um adversário ou até mesmo um jogador especifico que querem anular em uma partida. Tudo isso graças ao cruzamento de dados do histórico de desempenho dos atletas.

Outro uso da coleta desses dados está em beneficiar as grandes organizações com dados sociais, onde pode-se encontrar padrões e formar grupos dentro da base de fãs e executar campanhas em grupos-alvo. Conhecendo os fatores que mais atraem os fãs, a gestão das equipes pode se concentrar em melhorar esse aspecto, o que leva a ganhar novos fãs e reter os mais antigos. Os fãs podem ser alimentados com uma quantidade enorme de previsões e possibilidades em tempo real com as estatísticas do seu time ou jogador favorito onde podem comparar a sua performance com os outros jogadores no time rival ou do mesmo time.


A ciencia de dados tenta da sua maneira explorar as mesmas perguntas que os fãs e os treinadores se perguntam: quem eu acho que vai ganhar, e por quê? Quem são os melhores jogadores do mundo neste momento? Que estratégias são melhores, e porquê?.

<p align="center">
    <img width=750 src="https://github.com/DanOliveira-DS/NHL/blob/master/images/player.png?raw=true">
</p>

Mas é bom não confiar cegamente nas estatísticas. Por mais que o esporte americano seja excelente para criar uma base sólida de dados, o fator humano nunca pode ser deixado de lado. As estatísticas podem dizer algo, mas o contexto precisa ser levado em conta. Fatores que ainda não são representados nos dados, como a condição psicológica e o humor dos jogadores, também são chaves importantes na hora de analisar o jogo.

# Importando Bibliotecas


In [1]:
# suprimir avisos
import warnings
warnings.filterwarnings("ignore")

# bibliotecas para analise dos dados
import numpy as np # algebra linear
import pandas as pd # procesamento dos dados
import datetime

# setando opcoes globais do pandas
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

# bibliotecas para visualização dos dados
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

# Aquisição dos Dados
Os dados que usaremos nesse projeto foram coletados utilizando a **NHL stats API** e desponibilizados por Martin Ellis na plataforma Kaggle no formato *.csv*. Não são apenas os resultados e as estatísticas dos jogos da NHL, mas também detalhes de jogadas individuais, como chutes, gols e defesas, incluindo data, hora e coordenadas x e y. Ao total são 9 conjuntos de dados que se relacionam como apresentado na tabela abaixo:

<p align="center">
    <img width=900 src="https://github.com/DanOliveira-DS/NHL/blob/master/images/table_relationships.JPG?raw=true">
</p>


# Descrição dos conjuntos de dados:

* **game**: Reúne os registros de cada jogo no conjunto de dados;

* **game_golie_stats**: As estatísticas dos goleiros que participaram dos jogos;
  
* **game_plays**: As jogadas individuais de cada jogo;
    
* **game_plays_player**: Os jogadores envolvidos em uma jogada;
    
* **game_shifts**: Detalhes a respeito dos turnos dos jogadores;

* **game_skater_stats**: As estatísticas básicas dos jogadores. 36 entradas por jogo, uma para cada jogador do time;

* **game_teams_stats**: As estatísticas básicas das equipes. Cada jogo terá duas entradas nesta tabela, uma para cada equipe;

* **player_info**: Informações sobre os jogadores;
    
* **team_info**: informação sobre as equipes.

Caso esteja interessado em ver como os dados são coletados, indico o vídeo do [Kevin Sidwar](https://www.youtube.com/watch?v=avp7fvuMT1E&t=214s). Ele é também um dos responsaveis por dar inicio a documentação da NHL stats api

# Importando os conjuntos de dados

In [2]:
# nhl_eda
nhl_eda = pd.read_csv('datasets/nhl_eda_cleaned.csv')

# game_golie_stats
golie_stats = pd.read_csv('datasets/game_goalie_stats_cleaned.csv')

# game_skater_stats
skater_stats = pd.read_csv('datasets/game_skater_cleaned.csv')

# plays
plays = pd.read_csv('nhl_datasets/plays_cleaned.csv', chunksize=5000)


In [3]:
def load_dataset(df):
    
    # lista que vai armazenar os blocos
    chunk_list = []  

    # iterar sobre os blocos
    for chunk in df:
        chunk_list.append(chunk) # armazenando os blocos

    # concatenando os blocos
    return pd.concat(chunk_list)
    

In [4]:
plays = load_dataset(plays)

In [5]:
plays.head()

,play_id,game_id,play_num,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y,rink_side,player_id
0,2011030221_4,2011030221,4,4.0,1.0,Faceoff,NaN,0.0,0.0,1,REGULAR,0,1200,2012-04-29 19:20:13,0,0,Sean Couturier faceoff won against Patrik Elias,0.0,0.0,right,8476461
1,2011030221_4,2011030221,4,4.0,1.0,Faceoff,NaN,0.0,0.0,1,REGULAR,0,1200,2012-04-29 19:20:13,0,0,Sean Couturier faceoff won against Patrik Elias,0.0,0.0,right,8460542
2,2011030221_5,2011030221,5,4.0,1.0,Giveaway,NaN,28.0,24.0,1,REGULAR,21,1179,2012-04-29 19:21:02,0,0,Giveaway by Kimmo Timonen,-28.0,-24.0,right,8459670
3,2011030221_6,2011030221,6,4.0,1.0,Blocked Shot,NaN,52.0,28.0,1,REGULAR,26,1174,2012-04-29 19:21:07,0,0,Max Talbot blocked shot from Patrik Elias,-52.0,-28.0,right,8470171
4,2011030221_6,2011030221,6,4.0,1.0,Blocked Shot,NaN,52.0,28.0,1,REGULAR,26,1174,2012-04-29 19:21:07,0,0,Max Talbot blocked shot from Patrik Elias,-52.0,-28.0,right,8460542
